In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
# 엑셀 파일 경로
file_path = '성별_첫 취업 소요기간 및 평균소요기간(15 ~ 24년).xlsx'

In [3]:
# 1. 파일을 header 없이 불러오기 (병합 셀 구조 대응)
df_raw = pd.read_excel(file_path, sheet_name='데이터', header=None)

C:\Users\samsung\anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# 2. 1행 + 2행을 조합하여 열 이름 만들기
combined_header = df_raw.iloc[0].astype(str) + '_' + df_raw.iloc[1].astype(str)
combined_header.iloc[0] = '연령구분'
combined_header.iloc[1] = '성별'

In [5]:
# 3. 3행부터 데이터프레임 구성
df_data = df_raw[2:].copy()
df_data.columns = combined_header
df_data.reset_index(drop=True, inplace=True)

In [6]:
# 4. '15~29세 전체' 행 필터링
df_youth_total = df_data[
    (df_data['연령구분'] == '15~29세') & (df_data['성별'] == '전체')
].copy()

In [7]:
# 5. 연도 및 분석할 소요기간 구간 정의
years = list(range(2017, 2025))
period_labels = ['3개월 미만', '3~6개월 미만', '6개월~1년 미만',
                 '1~2년 미만', '2~3년 미만', '3년 이상']

In [8]:
# 6. 연도별 비율 계산
results = []

for year in years:
    try:
        # 임금근로자 수
        wage_col = f'{year}.05_임금 근로자'
        wage_total = pd.to_numeric(df_youth_total[wage_col].values[0], errors='coerce')

        if pd.isna(wage_total) or wage_total == 0:
            continue

        # 구간별 인원
        period_values = []
        for label in period_labels:
            col_key = f'{year}.05_{label}'
            val = pd.to_numeric(df_youth_total[col_key].values[0], errors='coerce')
            period_values.append(val)

        if any(pd.isna(val) for val in period_values):
            continue

        # 비율 계산
        period_percentages = [round(v / wage_total * 100, 2) for v in period_values]
        row = {'연도': year, '임금근로자수': int(wage_total)}
        for i, label in enumerate(period_labels):
            row[label] = period_percentages[i]
        results.append(row)
    except:
        continue

In [9]:
# 7. 데이터프레임 생성 및 엑셀 저장
df_result = pd.DataFrame(results)
df_result.to_excel('첫_취업_소요기간_임금근로자기준_비율_원본.xlsx', index=False)